In [1]:
import pandas as pd

In [2]:
HTTP = pd.read_json('..\BuiltDataset\HTTPDataSet_Standard.json','records')

E:\Anaconda3\envs\waf\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Starting with Pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only
  """Entry point for launching an IPython kernel.


In [3]:
HTTP

,payload,type
0,c/ caridad s/n,valid
1,"campello, el",valid
2,40184,valid
3,1442431887503330,valid
4,nue37,valid
...,...,...
31062,"<script a="">"" id=xss src=""http://ha.ckers.org/...",xss
31063,"<script a="">"" '' id=xss src=""http://ha.ckers.o...",xss
31064,"<script ""a='>'"" id=xss src=""http://ha.ckers.or...",xss
31065,"<script a=`>` id=xss src=""http://ha.ckers.org/...",xss


In [4]:
XSS = pd.read_json('..\BuiltDataset\XSSDataSet_Standard.json','records')

E:\Anaconda3\envs\waf\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Starting with Pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only
  """Entry point for launching an IPython kernel.


In [5]:
XSS

,payload,type
0,"<li><a href=""/wiki/file:socrates.png"" class=""i...",valid
1,"<tt onmouseover=""alert(1)"">test</tt>",xss
2,"</span> <span class=""reference-text"">steering ...",valid
3,"</span> <span class=""reference-text""><cite cla...",valid
4,"</span>. <a href=""/wiki/digital_object_identif...",valid
...,...,...
13681,<img onpointerenter=alert(1)>xss</img>,xss
13682,"<source onbeforepaste=""alert(1)"" contenteditab...",xss
13683,"<div draggable=""true"" contenteditable>drag me<...",xss
13684,"<li><cite id=""citerefdomingos2015"" class=""cita...",valid


In [6]:
Data = pd.concat([HTTP,XSS])

In [7]:
Data

,payload,type
0,c/ caridad s/n,valid
1,"campello, el",valid
2,40184,valid
3,1442431887503330,valid
4,nue37,valid
...,...,...
13681,<img onpointerenter=alert(1)>xss</img>,xss
13682,"<source onbeforepaste=""alert(1)"" contenteditab...",xss
13683,"<div draggable=""true"" contenteditable>drag me<...",xss
13684,"<li><cite id=""citerefdomingos2015"" class=""cita...",valid


In [8]:
Data = Data[~Data['type'].isin(['cmdi','path-traversal'])]

In [9]:
import numpy as np
X = Data['payload'].to_numpy().astype(str)
Y = Data['type'].to_numpy().astype(str)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

In [11]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2, random_state = 1, stratify = Y)

In [12]:
pipe = make_pipeline(TfidfVectorizer(input = 'content', lowercase = True, analyzer = 'char', max_features = 1024), SVC())
param_grid = {'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 4),(1,8)], 'svc__C': [1, 10], 'svc__kernel': ['linear', 'rbf']}
grid = GridSearchCV(pipe,param_grid,cv=2,verbose=10)
grid.fit(trainX,trainY)

Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV] svc__C=1, svc__kernel=linear, tfidfvectorizer__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  svc__C=1, svc__kernel=linear, tfidfvectorizer__ngram_range=(1, 1), score=0.994, total=   4.3s
[CV] svc__C=1, svc__kernel=linear, tfidfvectorizer__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


[CV]  svc__C=1, svc__kernel=linear, tfidfvectorizer__ngram_range=(1, 1), score=0.992, total=   4.0s
[CV] svc__C=1, svc__kernel=linear, tfidfvectorizer__ngram_range=(1, 2) 


...

[CV]  svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 1), score=0.998, total=   4.5s
[CV] svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 1) ..


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.3min remaining:    0.0s


[CV]  svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 1), score=0.997, total=   4.3s
[CV] svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 2) ..
[CV]  svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 2), score=0.999, total=  19.3s
[CV] svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 2) ..
[CV]  svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 2), score=0.998, total=  18.9s
[CV] svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 4) ..
[CV]  svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 4), score=0.998, total=  19.1s
[CV] svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 4) ..
[CV]  svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 4), score=0.998, total=  19.1s
[CV] svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 8) ..
[CV]  svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1, 8), score=0.998, total=  21.0s
[CV] svc__C=1, svc__kernel=rbf, tfidfvectorizer__ngram_range=(1

[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:  6.3min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidfvectorizer',
                                        TfidfVectorizer(analyzer='char',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=1024,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                               

In [13]:
grid.score(testX,testY)

0.9995492957746479

In [14]:
from sklearn.metrics import classification_report
Prediction = grid.predict(testX)
print(classification_report(testY,Prediction))

              precision    recall  f1-score   support

        sqli       1.00      1.00      1.00      2170
       valid       1.00      1.00      1.00      5124
         xss       1.00      1.00      1.00      1581

    accuracy                           1.00      8875
   macro avg       1.00      1.00      1.00      8875
weighted avg       1.00      1.00      1.00      8875



In [15]:
grid.best_params_

{'svc__C': 10, 'svc__kernel': 'rbf', 'tfidfvectorizer__ngram_range': (1, 2)}

In [16]:
pipe = make_pipeline(TfidfVectorizer(input = 'content', lowercase = True, analyzer = 'char', max_features = 1024, ngram_range = (1, 2)), SVC(C = 10, kernel = 'rbf'))
pipe.fit(trainX,trainY)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=1024,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=10, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='

In [17]:
pipe.score(testX,testY)

0.9995492957746479

In [18]:
Prediction = grid.predict(testX)
print(classification_report(testY,Prediction))

              precision    recall  f1-score   support

        sqli       1.00      1.00      1.00      2170
       valid       1.00      1.00      1.00      5124
         xss       1.00      1.00      1.00      1581

    accuracy                           1.00      8875
   macro avg       1.00      1.00      1.00      8875
weighted avg       1.00      1.00      1.00      8875



In [19]:
import joblib

joblib.dump(pipe, 'TF-IDF_SVM_V1.0.m')
joblib.dump(pipe, 'TF-IDF_SVM_V1.0.joblib')

['TF-IDF_SVM_V1.0.joblib']